In [ ]:
import math
import time

import numpy as np
import pandas as pd

from PIL import Image

from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.models import Sequential
from keras.utils import np_utils

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

In [ ]:
def cnn_model(kernel_size, nb_filters, nb_classes, input_shape):
    """
    Define and run the Convolutional Neural Network

    INPUT
        kernel_size: Initial size of kernel
        nb_filters: Initial number of filters
        nb_classes: Number of classes for classification

    OUTPUT
        Fitted CNN model
    """

    model = Sequential()

    model.add(Conv2D(nb_filters, 
                     (kernel_size[0], kernel_size[1]),
                     padding='valid',
                     strides=1,
                     input_shape=input_shape,
                     activation="relu"))

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]), activation="relu"))

    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]), activation="relu"))

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    print("Model flattened out to: ", model.output_shape)

    model.add(Dense(128))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.25))

    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])


    return model

In [ ]:
def save_model(model, score, model_name):
    """
    Saves Keras model to an h5 file, based on precision_score

    INPUT
        model: Keras model object to be saved
        score: Score to determine if model should be saved.
        model_name: name of model to be saved
    """

    print("Saving Model")
    model.save("../models/" + model_name + "_recall_" + str(round(score, 4)) + ".h5")

In [ ]:
batch_size = 100
    nb_classes = 2
    nb_epoch = 25

    img_rows, img_cols = 256, 256
    channels = 3
    nb_filters = 32
    kernel_size = (8, 8)

    input_shape = (img_rows, img_cols, channels)

    print("Training Model")
    stop = EarlyStopping(monitor='val_acc',
                         min_delta=0.001,
                         patience=2,
                         verbose=0,
                         mode='auto')

    filepath = "../checkpoints/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    
    model = cnn_model(kernel_size, nb_filters, nb_classes, (img_rows, img_cols, channels))
    print(model.summary())
    
    model.fit_generator(generator=train_generator,
                        epochs=nb_epoch,
                        validation_data=val_generator,
                        callbacks=[tensor_board, checkpoint],
                        class_weight='auto',
                        verbose=2,
                        use_multiprocessing=False)